In [53]:
from opentrons.simulate import get_protocol_api

import json 
import pandas as pd 
from datetime import datetime
from pytz import timezone 
Pacific = timezone('US/Pacific')
from IPython.display import clear_output
import time

In [54]:
START_WELL = "A1"
volume_df = pd.read_csv('./misc/repeat_96_well_plate.csv')
volume_df

,CTAB,Water,GCH,AA,SN,GS
0,96,110,59,9,2,24
1,96,98,59,6,17,24
2,96,95,59,9,17,24
3,96,103,59,11,7,24
4,96,99,59,13,9,24
5,96,95,59,15,11,24
6,96,106,59,13,2,24
7,96,86,59,29,6,24
8,96,83,59,29,9,24
9,96,79,59,29,13,24


In [55]:
protocol = get_protocol_api('2.18')
CUSTOM_LABWARE_PATH = "../opentrons_custom_labware/"
protocol.home()

/Users/pozzolabadmin/.opentrons/robot_settings.json not found. Loading defaults
Deck calibration not found.
/Users/pozzolabadmin/.opentrons/deck_calibration.json not found. Loading defaults


In [56]:
protocol.set_rail_lights(True)

In [57]:
# load all the labware modules
tiprack_300 = protocol.load_labware(
    load_name="opentrons_96_tiprack_300ul",
    location=7)

p300 = protocol.load_instrument(
    instrument_name="p300_single_gen2",
    mount="right",
    tip_racks=[tiprack_300]
    )

tiprack_20 = protocol.load_labware(
    load_name="opentrons_96_tiprack_20ul",
    location=4)

p20 = protocol.load_instrument(
    instrument_name="p20_single_gen2",
    mount="left",
    tip_racks=[tiprack_20]
    )

with open(CUSTOM_LABWARE_PATH+'20mlscintillation_12_wellplate_18000ul.json') as labware_file:
    stocks_def = json.load(labware_file)
    stocks = protocol.load_labware_from_definition(stocks_def, location=1)

temp_module = protocol.load_module(module_name="temperature module gen2", location=6)
alplate_adapter = temp_module.load_adapter("opentrons_aluminum_flat_bottom_plate")
plate = alplate_adapter.load_labware('corning_96_wellplate_360ul_flat')
temp_module.set_temperature(celsius=30)

In [58]:
tiprack_300.set_offset(x=0.50, y=1.10, z=0.00)
tiprack_20.set_offset(x=0.00, y=0.90, z=0.00)
plate.set_offset(x=0.00, y=1.50, z=0.00)

In [59]:
tiprack_20_wells = [well for row in tiprack_20.rows() for well in row]
tiprack_300_wells = [well for row in tiprack_300.rows() for well in row]
stocks_wells = [well for row in stocks.rows() for well in row]
plate_wells = [well for row in plate.rows() for well in row]

In [60]:
START_WELL_INDEX = next((i for i, well in enumerate(plate_wells) if well.well_name == START_WELL), None)
if START_WELL_INDEX is not None:
    print(f"The index of well {START_WELL} is {START_WELL_INDEX}.")
else:
    print(f"Well {START_WELL} not found.")

The index of well A1 is 0.


In [61]:
def synthesize(stock_index, ds):
    """ Synthesize AuNP by mixing components

    stock_index : index of the stock to add (int)
    ds : a pandas dataseries with volumes to be added. 
    """
    p300.pick_up_tip(tiprack_300_wells[int(stock_index)])
    has_used_p20, has_used_p300 = False, False
    need_p20 = (ds<20).any()
    if need_p20:
        p20.pick_up_tip(tiprack_20_wells[int(stock_index)])
    for index, value in ds.items():
        if value>20:
            pipette = p300 
            has_used_p300 = True
        else:
            pipette = p20
            has_used_p20 = True
        source_well = stocks_wells[int(stock_index)]
        target_well = plate_wells[int(index) + START_WELL_INDEX]
        current_date_time = datetime.now(Pacific)
        time_str = current_date_time.strftime('%H:%M:%S')
        print("[%s] Dispensing %s of %d from %s into well %s "%(time_str, ds.name, value, source_well.well_name, target_well.well_name)) #,end='\r', flush=False)
        pipette.aspirate(value, source_well)
        pipette.dispense(value, target_well.top())
        if not stock_index in [0, 5]:
            pipette.blow_out()
        if stock_index==1:
            has_used_p300 = False
            
    if has_used_p20:
        p20.drop_tip()
        
    if has_used_p300:
        p300.drop_tip()
    else:
        p300.return_tip()

In [69]:
for _ in range(2):
    synth_volumes_df = volume_df.sample(frac=1)
    for stock_index,(_, stock_vol_series) in enumerate(synth_volumes_df.items()):
        synthesize(stock_index, stock_vol_series)

# Add water for control
p300.pick_up_tip(tiprack_300_wells[1])
source_well = stocks_wells[1]
p300.aspirate(300, stocks_wells[1])
target_well = plate_wells[len(volume_df) + START_WELL_INDEX]
p300.dispense(200, target_well.top())
current_date_time = datetime.now(Pacific)
time_str = current_date_time.strftime('%H:%M:%S')
print("[%s] Dispensing %s of %d from %s into well %s "%(time_str, "water", 300, source_well.well_name, target_well.well_name))
p300.drop_tip()


[14:45:50] Dispensing CTAB of 96 from A1 into well A9 
[14:45:50] Dispensing CTAB of 96 from A1 into well A3 
[14:45:50] Dispensing CTAB of 96 from A1 into well A5 
[14:45:50] Dispensing CTAB of 96 from A1 into well A1 
[14:45:50] Dispensing CTAB of 96 from A1 into well A4 
[14:45:50] Dispensing CTAB of 96 from A1 into well A6 
[14:45:50] Dispensing CTAB of 96 from A1 into well A7 
[14:45:50] Dispensing CTAB of 96 from A1 into well A8 
[14:45:50] Dispensing CTAB of 96 from A1 into well A2 
[14:45:50] Dispensing CTAB of 96 from A1 into well A10 
[14:45:50] Dispensing Water of 83 from A2 into well A9 
[14:45:50] Dispensing Water of 95 from A2 into well A3 
[14:45:50] Dispensing Water of 99 from A2 into well A5 
[14:45:50] Dispensing Water of 110 from A2 into well A1 
[14:45:50] Dispensing Water of 103 from A2 into well A4 
[14:45:50] Dispensing Water of 95 from A2 into well A6 
[14:45:50] Dispensing Water of 106 from A2 into well A7 
[14:45:50] Dispensing Water of 86 from A2 into well A8

<InstrumentContext: p300_single_v2.1 in RIGHT>

In [64]:
current_date_time = datetime.now(Pacific)
print(current_date_time.strftime('End Time: %H:%M:%S'))

End Time: 14:44:23


In [65]:
protocol.set_rail_lights(False)

In [66]:
protocol.home()

In [67]:
print("Keeping the temperature module on for 30 mins at the initial temperature")

Keeping the temperature module on for 30 mins at the initial temperature


In [68]:
def countdown(minutes):
    total_seconds = minutes * 60
    for remaining in range(total_seconds, 0, -1):
        # Convert seconds to minutes and seconds
        mins, secs = divmod(remaining, 60)
        # Clear output in the notebook for a clean update
        clear_output(wait=True)
        print(f"Time remaining: {mins:02d}:{secs:02d}")
        time.sleep(1)
    clear_output(wait=True)
    print("Time's up!")

# Start a 30-minute countdown
countdown(30)


Time remaining: 29:56


KeyboardInterrupt: 